In [285]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from geopy import distance
import statsmodels.api as sm
from stargazer.stargazer import Stargazer
from IPython.core.display import HTML
import linearmodels as lm
from linearmodels import PanelOLS
from linearmodels.panel import compare
import holoviews as hv
hv.config(future_deprecations=True)
import hvplot.pandas
import plotly.io as pio
pio.renderers
pio.renderers.default = "notebook_connected"
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import cufflinks
cufflinks.go_offline()
cufflinks.set_config_file(world_readable=True, theme='pearl')
import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly
from plotly import tools
import kaleido
import plotly.express as px
init_notebook_mode(connected=True)
center = (39.952,-75.164)

In [286]:
with open("../output/cbd_census.csv", "r") as cen:
    df_c = pd.read_csv(cen)
with open("../output/bl_census_tract.csv", "r") as bl:
    df_bl = pd.read_csv(bl)
with open("../output/missing_census_tract.csv", "r") as missing:
    df_makeup = pd.read_csv(missing)
cbd_census = df_c.tract.unique()

In [287]:
with open('../output/pop.csv', "r") as p:
    pop = pd.read_csv(p)
    
shape_map = gpd.read_file('../data/Philadelphia_Census_Tracts_2010/Philadelphia_Census_Tracts_2010.shp')

gdf = shape_map.to_crs(epsg=4326)  
area = gdf[['TRACTCE10','SHAPE_AREA']]
area['tr'] = area.TRACTCE10.astype(float)/100

l = [pop]
for y in range(1993,2009):
    df = pop[pop.yr==2009]
    df['yr'] = y
    l.append(df)
for y in range(2020,2023):
    df = pop[pop.yr==2019]
    df['yr'] = y
    l.append(df)
pop = pd.concat(l,ignore_index=True)
pop=pop.drop_duplicates(['tr','yr'])
# pop['tr'] = pop.tr*100
# area['tr'] = area.tr*100
pop=pop[pop.yr>=2009]

d = {'tr': np.repeat(pop.tr.unique(),14), 'yr': np.tile(range(2009,2023), 535)}
df_pop=pd.DataFrame(d)
df_pop= df_pop.merge(pop, how='left',on=['tr','yr'])
df_pop["population"] = df_pop.groupby("tr").transform(lambda x: x.fillna(x.mean()))['population']
pop_ct=pd.DataFrame(df_pop.groupby('tr').mean()['population']).reset_index()
den_df= pop_ct.merge(area,on='tr', how ='right')
den_df['density']=den_df['population']*1000000/den_df['SHAPE_AREA']
den_df['density_rank'] = den_df.density.rank(pct = True)
den_df['density_group'] = den_df.apply(lambda x: 'high_density' if  x['density_rank']>=0.9 else 'mid_density' if (x['density_rank']<0.9 and x['density_rank']>=0.5) else 'low_density', axis=1)



In [282]:
den_df.to_csv('../output/den.csv',index=False)

In [288]:
shape_map_path = r"../data/business_licenses/business_licenses.shp"
shape_map = gpd.read_file(shape_map_path)
license = shape_map.to_crs(epsg=4326)  # EPSG 4326 = WGS84 = https://epsg.io/4326
food_l = license.licensetyp.str.contains('Food|Cafe', case=False)
food_license = license[food_l]
food_license.loc[:, 'longitude'] = food_license.geometry.x
food_license.loc[:, 'latitude'] = food_license.geometry.y
food_license['open'] = food_license.initialiss.str[:4]
food_license['close'] = food_license.inactiveda.str[:4]
food_license = food_license[(food_license.latitude.notna()) & (food_license.longitude.notna())]
food_license['distance'] = food_license.apply(lambda x: distance.distance((x['latitude'], x['longitude']), center).km, axis=1) 
food_license['is_cbd'] = food_license.apply(lambda x: 1 if x['distance'] <= 2 else 0, axis=1) 
food_license.reset_index(inplace=True, drop=True)
# food_license['c_mer'] = food_license.censustrac.astype('float')
# food_license = food_license.merge(pop, left_on='c_mer', right_on='tr',how='left')

In [289]:
df_pop['tr'] = df_pop.tr*100
area['tr'] = area.tr*100

In [290]:
df_use = food_license[['longitude','latitude','censustrac' , 'open', 'close', 'is_cbd']]
df_cen = pd.DataFrame(df_use.censustrac.value_counts())
df_cen.reset_index(inplace=True)
df_cen.rename({"index":"census_track", "censustrac": "count"}, inplace=True, axis=1)
df_use['close'][df_use.close=='3013'] = '2013'
missing={'369':'369.02', '9800':'9800.03', '1': '1.01', '4': '4.03', '7':'701', '8': '8.5', '12':'12.04', '13':'13.01','125':'125.01'}
#df_use['censustrac'] = df_use.apply(lambda x: missing[x['censustrac']] if x['censustrac'] in missing else x['censustrac'], axis=1)
cen_trac = df_use.censustrac.unique()

df_use=df_use[df_use.open.astype('int')>=2013]
df_use=df_use[df_use.open.astype('int')!=2022]

df_use['post_covid'] = df_use.apply(lambda x: 1 if int(x.open)>=2020 else 0, axis=1)
df_use.reset_index(inplace=True)

df_bl = df_bl.drop_duplicates(['lng','lat'])
df_bl= df_bl[df_bl.tract.notna()]
df_bl=df_bl[['lat','lng','tract']]
df_bl.reset_index(inplace=True,drop=True)

df_new = pd.merge(df_use, df_bl, left_on=['longitude','latitude'], right_on=['lng','lat'], how='left')
df_new= df_new.round({'latitude':5,'longitude':5})

df_makeup = df_makeup.round({'lat':5,'lng':5})
df_makeup.rename({'tract':'tr'},inplace=True,axis=1)

df_new= df_new[['latitude','longitude','censustrac','tract','open','close','is_cbd','post_covid']]
df_full = pd.merge(df_new, df_makeup, left_on=['latitude','longitude'], right_on=['lat','lng'], how='left')

In [291]:
df_full['tract'] = df_full.apply(lambda x: x.tr if pd.isna(x.tract) else x.tract, axis=1)
df_full['censustract'] = (df_use['censustrac'].astype('float')*100).astype('int')
df_full['tract'] = df_full.apply(lambda x: x.censustract if pd.isna(x.tract) else x.tract, axis=1)
base = df_full[df_full.open.astype('int')<=2018][(df_full.close.isna())|(df_full.close[df_full.close.notna()].astype('int')>=2019)]
base=base.groupby('tract').count().reset_index()[['tract','longitude']]
base.rename({'longitude':'count'},axis=1,inplace=True)

In [309]:
df=df_full.groupby(['tract','open']).count().reset_index()[['tract','open','is_cbd']]
df= df.merge(base,on='tract',how='left')
df.rename({'open':'year','is_cbd':'New open'},inplace=True,axis=1)

df['is_cbd'] = df.apply(lambda row: 1 if row.tract in cbd_census else 0,axis =1)
df['post_covid'] = df.apply(lambda x: 1 if int(x.year)>=2019 else 0, axis=1)
df['DID'] = df.apply( lambda x: 1 if (x['is_cbd']==1) and (x['post_covid']==1) else 0, axis=1)
# create log open count
df['open_ln'] = np.log(df['New open'])
census_dic = dict(df.tract.value_counts())
df['year']= df.year.astype(int)

df = df.merge(df_pop,left_on = ['tract','year'], right_on=['tr','yr'], how='left')
df = df.merge(area,left_on = ['tract'], right_on=['tr'], how='left')
df=df[df['population']!=0]
df['pop'] = df.population/1000
df['density'] = df.population*1000/df.SHAPE_AREA
df['post covid * density'] = df['density']  * df['post_covid'] 

df['density_rank'] = df.density.rank(pct = True)
df['density_group'] = df.apply(lambda x: 'high_density' if  x['density_rank']>=0.9 else 'mid_density' if (x['density_rank']<0.9 and x['density_rank']>=0.5) else 'low_density' if (x['density_rank']<0.5) else np.NAN , axis=1)
y_l  =sorted(df.year.unique())

#df.to_csv('../output/bl_ct_2.csv', index=False)

# for k in df['tract'].unique():
#     if census_dic[k] <= 5: #10
#         df.drop(df[df['tract'] == k].index, inplace = True)
#df = df[df['New open']<50]

df['open_rate']=df['New open']/df['count']
df_2019_o=df[df.year==2020][['tract','open_rate']].reset_index(drop=True)
df = df.merge(df_2019_o, on='tract', how='left')
df['open_rate_d'] = df['open_rate_x'] -df['open_rate_y']

cbd = [df[df.year==y].groupby('is_cbd')['open_rate_d'].mean()[1] for y in y_l]
high = [df[df.year==y].groupby('density_group')['open_rate_d'].mean()[0] for y in y_l]
mid = [df[df.year==y].groupby('density_group')['open_rate_d'].mean()[2] for y in y_l]
low = [df[df.year==y].groupby('density_group')['open_rate_d'].mean()[1] for y in y_l]

In [310]:
years=[y-1 for y in y_l]
fig = go.Figure()
fig.add_trace(go.Scatter(x=years, y=cbd,
                    mode='lines+markers',
                    name='CBD'))
fig.add_trace(go.Scatter(x=years, y=high,
                    mode='lines+markers',
                    name='high density'))
fig.add_trace(go.Scatter(x=years, y=mid,
                    mode='lines+markers',
                    name='mid density'))
fig.add_trace(go.Scatter(x=years, y=low,
                    mode='lines+markers',
                    name='low density'))
fig.add_vrect(x0=2019, x1=2021, 
              annotation_text="decline", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Times New Roman"),
              fillcolor="green", opacity=0.25, line_width=0)

fig.update_layout(
    autosize=False,
    width=800,
    height=600,yaxis_title='open rate')

fig.show()


In [311]:
import plotly.express as px

df['cbd'] = df.apply(lambda x: 'CBD' if x.is_cbd==1 else 'non-CBD', axis=1)
fig = go.Figure()
fig.add_trace(go.Box(
    y=df[df.post_covid==0]['open_rate_d'],
    x=df.cbd,
    name='before covid'
))
fig.add_trace(go.Box(
    y=df[df.post_covid==1]['open_rate_d'],
    x=df.cbd,
    name='post covid'
))
fig.update_layout(autosize=False,
    width=800,
    height=600,
    yaxis_title='open_rate_d',
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.show()

In [328]:
df1=df_full.groupby(['tract','close']).count().reset_index()[['tract','close','is_cbd']]
df1= df1.merge(base,on='tract',how='left')
df1.rename({'close':'year','is_cbd':'Close-down'},inplace=True,axis=1)
df1=df1[df1.year.astype('int')!=2022]
df1['is_cbd'] = df1.apply(lambda row: 1 if row.tract in cbd_census else 0,axis =1)
df1['post_covid'] = df1.apply(lambda x: 1 if int(x.year)>=2019 else 0, axis=1)
df1['DID'] = df1.apply( lambda x: 1 if (x['is_cbd']==1) and (x['post_covid']==1) else 0, axis=1)
# create log open count
df1['close_ln'] = np.log(df1['Close-down'])
census_dic = dict(df1.tract.value_counts())
df1['year']= df1.year.astype(int)

df1 = df1.merge(df_pop,left_on = ['tract','year'], right_on=['tr','yr'], how='left')
df1 = df1.merge(area,left_on = ['tract'], right_on=['tr'], how='left')
df1=df1[df1['population']!=0]
df1['pop'] = df1.population/1000
df1['density'] = df1.population*1000/df.SHAPE_AREA
df1 = df1[df1.year!=2012]
df1['post covid * density'] = df['density']  * df['post_covid'] 

df1['density_rank'] = df1.density.rank(pct = True)
df1['density_group'] = df1.apply(lambda x: 'high_density' if  x['density_rank']>=0.9 else 'mid_density' if (x['density_rank']<0.9 and x['density_rank']>=0.5) else 'low_density' if (x['density_rank']<0.5) else np.NAN , axis=1)
y_l  =range(2014,2022)

# for k in df['tract'].unique():
#     if census_dic[k] <= 5: #10
#         df.drop(df[df['tract'] == k].index, inplace = True)
#df = df[df['New open']<50]

df1['close_rate']=df1['Close-down']/df1['count']
df_2019_o=df1[df1.year==2019][['tract','close_rate']].reset_index(drop=True)
df1 = df1.merge(df_2019_o, on='tract', how='left')
df1['close_rate_d'] = df1['close_rate_x'] -df1['close_rate_y']
df1=df1[df1.density_group.notna()]
cbd = [df1[df1.year==y].groupby('is_cbd')['close_rate_d'].mean()[1] for y in y_l]
high = [df1[df1.year==y].groupby('density_group')['close_rate_d'].mean()[0] for y in y_l]
mid = [df1[df1.year==y].groupby('density_group')['close_rate_d'].mean()[2] for y in y_l]
low = [df1[df1.year==y].groupby('density_group')['close_rate_d'].mean()[1] for y in y_l]

In [330]:
years=[y for y in y_l]
fig = go.Figure()
fig.add_trace(go.Scatter(x=years, y=cbd,
                    mode='lines+markers',
                    name='CBD'))
fig.add_trace(go.Scatter(x=years, y=high,
                    mode='lines+markers',
                    name='high density'))
fig.add_trace(go.Scatter(x=years, y=mid,
                    mode='lines+markers',
                    name='mid density'))
fig.add_trace(go.Scatter(x=years, y=low,
                    mode='lines+markers',
                    name='low density'))
fig.add_vrect(x0=2019, x1=2021, 
              annotation_text="decline", annotation_position="top left",
              annotation=dict(font_size=20, font_family="Times New Roman"),
              fillcolor="green", opacity=0.25, line_width=0)

fig.update_layout(
    autosize=False,
    width=800,
    height=600,yaxis_title='close rate')

fig.show()


In [332]:
import plotly.express as px

df1['cbd'] = df1.apply(lambda x: 'CBD' if x.is_cbd==1 else 'non-CBD', axis=1)
fig = go.Figure()
fig.add_trace(go.Box(
    y=df1[df1.post_covid==0]['close_rate_d'],
    x=df1.cbd,
    name='before covid'
))
fig.add_trace(go.Box(
    y=df1[df1.post_covid==1]['close_rate_d'],
    x=df1.cbd,
    name='post covid'
))
fig.update_layout(autosize=False,
    width=800,
    height=600,
    yaxis_title='close_rate_d',
    boxmode='group' # group together boxes of the different traces for each value of x
)
fig.show()

In [ ]:
fig.write_image("../output/bl_open.png", engine='kaleido')

In [ ]:
fig.write_image("../output/bl_density.png", engine='kaleido')

In [333]:
df['interval'] = df['year'] - 2013
df['interval * Covid'] = df['interval']  * df['post_covid']

# fit model 
x = df.loc[:,['interval', 'post_covid','interval * Covid']]
y = df['New open']
X2 = sm.add_constant(x)
est_its = sm.OLS(y, X2).fit(cov_type='HC2', use_t=None)
# fit model 
x = df[df.is_cbd==1].loc[:,['interval', 'post_covid','interval * Covid']]
y = df[df.is_cbd==1]['New open']
X2 = sm.add_constant(x)
est_its_cdb = sm.OLS(y, X2).fit(cov_type='HC2', use_t=None)

# fit model 
x = df.loc[:,['is_cbd', 'post_covid','DID']]
y = df[df['New open'].notna()]['New open']
X2 = sm.add_constant(x)
est_o = sm.OLS(y, X2).fit(cov_type='HC2', use_t=None)

stargazer = Stargazer([est_o,est_its,est_its_cdb])
stargazer.title('Regression results')
stargazer.custom_columns(['New Open','ITS','ITS-CBD'], [1, 1,1])
stargazer.significant_digits(2)
stargazer.covariate_order(['DID', 'post_covid', 'is_cbd','interval','interval * Covid','const'])
stargazer.rename_covariates({'DID': 'Post Covid * CBD', 'is_cbd': 'CBD',
                              'const':'Constant','post_covid':'Post Covid'})
stargazer.show_degrees_of_freedom(False)
stargazer.add_custom_notes(['Data Source: Business Licenses - Philadelphia (2022)'])
stargazer.cov_spacing = 2
stargazer

In [334]:
stargazer = Stargazer([est_its,est_its_cdb])
stargazer.title('Regression results')
stargazer.custom_columns(['Inflows(ITS)','Inflows(ITS-CBD)'], [1, 1])
stargazer.significant_digits(2)
stargazer.covariate_order([ 'interval * Covid','post_covid','interval','const'])
stargazer.rename_covariates({
                              'const':'Constant','post_covid':'Post Covid'})
stargazer.show_degrees_of_freedom(False)
stargazer.add_custom_notes(['Data Source: Business Licenses - Philadelphia (2022)'])
stargazer.cov_spacing = 2
stargazer

In [335]:
df['year']=df['year'].astype(int)
df['year_index'] = df['year']
df['tract_index'] = df['TRACTCE10']
df.set_index(['tract_index','year_index'],inplace=True)

mod1 = PanelOLS(df['New open'],sm.add_constant(df[['DID','density']]),entity_effects=True, time_effects=True)
clfe1 = mod1.fit(cov_type = 'clustered', cluster_entity = True)
# mod2 = PanelOLS(df[df['open_ln']>0]['open_ln'], sm.add_constant(df[df['open_ln']>0][['DID','density']]), entity_effects=True, time_effects=True)
# clfe2 = mod2.fit(cov_type = 'clustered', cluster_entity = True)

print(compare({'mod1': clfe1}, stars = True))

           Model Comparison          
                                 mod1
-------------------------------------
Dep. Variable                New open
Estimator                    PanelOLS
No. Observations                 2618
Cov. Est.                   Clustered
R-squared                      0.0009
R-Squared (Within)            -0.0004
R-Squared (Between)            0.0008
R-Squared (Overall)            0.0086
F-statistic                    1.0501
P-value (F-stat)               0.3501
=====================     ===========
const                       4.1542***
                             (5.8066)
DID                            0.5102
                             (1.1037)
density                        0.0230
                             (0.0229)
======================= =============
Effects                        Entity
                                 Time
-------------------------------------

T-stats reported in parentheses


In [336]:
print(clfe1.summary)

                          PanelOLS Estimation Summary                           
Dep. Variable:               New open   R-squared:                        0.0009
Estimator:                   PanelOLS   R-squared (Between):              0.0008
No. Observations:                2618   R-squared (Within):              -0.0004
Date:                Sun, May 22 2022   R-squared (Overall):              0.0086
Time:                        16:56:11   Log-likelihood                   -6042.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      1.0501
Entities:                         349   P-value                           0.3501
Avg Obs:                       7.5014   Distribution:                  F(2,2259)
Min Obs:                       1.0000                                           
Max Obs:                       9.0000   F-statistic (robust):             0.7330
                            

In [337]:
placebo_df = data[(data['post_covid'] ==0)]
# Transform dataframe to match the equation
placebo_df['fake_after1'] = placebo_df['year'] > 2018
placebo_df['fake_after2'] = placebo_df['year'] > 2016
placebo_df['fake_treated1'] = 1*(placebo_df['is_cbd'] & placebo_df['fake_after1'])
placebo_df['fake_treated2'] = 1*(placebo_df['is_cbd'] & placebo_df['fake_after2'])

mod1 = lm.PanelOLS.from_formula(''' new_open ~ 
fake_treated1 + EntityEffects + TimeEffects''',placebo_df)
mod2 = lm.PanelOLS.from_formula(''' new_open ~ 
fake_treated2 + EntityEffects + TimeEffects''',placebo_df)
# Specify clustering when we fit the model
clfe1 = mod1.fit(cov_type = 'clustered', cluster_entity = True)
clfe2 = mod2.fit(cov_type = 'clustered', cluster_entity = True)
print(clfe1)
print(clfe2)

NameError: name 'data' is not defined